# Step 1: Preprocessing Data for Training
Missing values are imputed

In [1]:
import pandas as pd
import numpy as np

In [2]:
datafile_name = "EDA_df.csv"
data_filepath = f'./Raw_Data/{datafile_name}'
basis_df = pd.read_csv(data_filepath)

In [3]:
basis_df.columns

Index(['application_number', 'customer_no', 'product_description',
       'loan_account_no', 'requested_loan_amount', 'original_emi_amount',
       'res_years_at_current_city', 'res_resident_status', 'res_city',
       'res_state', 'age', 'date_of_birth', 'manufacturer',
       'down_payment_amount', 'total_income', 'branch_name', 'occupation_type',
       'industry', 'interest_rate', 'number_of_installments',
       'loan_asset_cost', 'bureau_score', 'asset_category', 'asset_model',
       'dealer', 'branch_id', 'channel', 'final_sanctioned_amount',
       'disbursal_amount', 'res_years_at_current_address', 'gender',
       'marital_status', 'nature_of_business', 'years_in_occupation',
       'loan_application_date'],
      dtype='object')

In [4]:
basis_df.isnull().sum()

application_number                  0
customer_no                         0
product_description                 0
loan_account_no                     0
requested_loan_amount               0
original_emi_amount                 0
res_years_at_current_city           0
res_resident_status                 1
res_city                            4
res_state                           0
age                                 0
date_of_birth                       0
manufacturer                        0
down_payment_amount                 0
total_income                        2
branch_name                         0
occupation_type                     1
industry                          360
interest_rate                       0
number_of_installments              0
loan_asset_cost                     0
bureau_score                        0
asset_category                      0
asset_model                         0
dealer                              0
branch_id                           0
channel     

In [5]:
#load data
datafile_name = datafile_name
data_filepath = f'./Raw_Data/{datafile_name}'
basis_df = pd.read_csv(data_filepath)

#lower case all column names
basis_df.columns = basis_df.columns.str.lower()

#drop missing and zero disbursal_amount      #reason: no disbursal amount
# basis_df = basis_df.dropna(subset = 'disbursal_amount')
# basis_df = basis_df[basis_df.disbursal_amount != 0]

#choose necessary model variables
variables = ['bureau_score', 'product_description','loan_account_no',
             'res_years_at_current_city','res_resident_status','res_city',
             'res_state','age','manufacturer',
             'down_payment_amount','total_income','branch_name',
             'occupation_type', 'industry','interest_rate',
             'number_of_installments','loan_asset_cost','asset_model',
             'channel','res_years_at_current_address','gender',
             'marital_status', 'nature_of_business','years_in_occupation']
#note: no requested_loam_amount (to be confirmed)

df = basis_df[variables].copy()

In [6]:
df = df[df['product_description'] != 'Used Two-Wheeler']
df = df[df['total_income'] >= 1000]
df = df[df['loan_account_no'] != 'TW01010103000002258833']

In [7]:
df.shape

(49681, 24)

In [8]:
print(f'res_resident_status mode: {df.res_resident_status.mode()[0]}')
print(f'res_city mode: {df.res_city.mode()[0]}')
print(f'total_income median: {df.total_income.median()}')
print(f'occupation_type mode: {df.occupation_type.mode()[0]}')
print(f'industry mode: {df.industry.mode()[0]}')
print(f'nature_of_business mode: {df.nature_of_business.mode()[0]}')
print(f'years_in_occupation median: {df.years_in_occupation.median()}')

res_resident_status mode: Owned by Parent /Sibling
res_city mode: MUMBAI
total_income median: 420000.0
occupation_type mode: Salaried
industry mode: Others
nature_of_business mode: Services
years_in_occupation median: 3.0


In [9]:
def standard_format_data(raw_datafilename):
    #load necessary libraries
    import pandas as pd
    import numpy as np
    
    #load data
    datafile_name = raw_datafilename
    data_filepath = f'./Raw_Data/{datafile_name}'
    basis_df = pd.read_csv(data_filepath)

    #lower case all column names
    basis_df.columns = basis_df.columns.str.lower()

    #choose necessary model variables
    variables = ['bureau_score', 'product_description','loan_account_no',
             'res_years_at_current_city','res_resident_status',
             'res_city','res_state','age',
             'manufacturer','down_payment_amount','total_income',
             'branch_name','occupation_type', 'industry',
             'interest_rate','number_of_installments','loan_asset_cost',
             'asset_model','channel','res_years_at_current_address',
             'gender', 'marital_status', 'nature_of_business',
             'years_in_occupation']
    #note: no requested_loam_amount (to be confirmed)
    df = basis_df[variables].copy()

    df = df[df['product_description'] != 'Used Two-Wheeler']
    df = df[df['total_income'] >= 1000]
    df = df[df['loan_account_no'] != 'TW01010103000002258833']

    #fill missing values; imputation using median values
    #cols cannot have missing values: bureau_score
    df['total_income'] = df['total_income'].fillna(df.total_income.median())
    df['monthly_income'] = df['total_income'] / 12 #derived
    df['down_payment_amount'] = df['down_payment_amount'].fillna(df.down_payment_amount.median())
    df['loan_asset_cost'] = df['loan_asset_cost'].fillna(df.loan_asset_cost.median())
    df['dp_ratio'] = df.down_payment_amount/ df.loan_asset_cost #derived

    df['res_resident_status'] = df.res_resident_status.fillna(df.res_resident_status.mode()[0])
    df['res_city'] = df['res_city'].fillna(df.res_city.mode()[0])
    df['occupation_type'] = df['occupation_type'].fillna(df.occupation_type.mode()[0])
    df['industry'] = df['industry'].fillna(df.industry.mode()[0])
    df['nature_of_business'] = df['nature_of_business'].fillna(df.nature_of_business.mode()[0])
    df['years_in_occupation'] = df['years_in_occupation'].fillna(df.years_in_occupation.median())
    
    df['branch_name'] = df['branch_name'].fillna(df.branch_name.mode()[0])
    df['age'] = df['age'].fillna(df.age.median())
    df['number_of_installments'] = df['number_of_installments'].fillna(df.number_of_installments.median())
    df['res_state'] = df['res_state'].fillna(df.res_state.mode()[0])
    df['product_description'] = df['product_description'].fillna(df.product_description.mode()[0])
    df['marital_status'] = df['marital_status'].fillna(df.marital_status.mode()[0])
    df['manufacturer'] = df['manufacturer'].fillna(df.manufacturer.mode()[0])
    
    #standardize values capitalization
    df = df.map(lambda x: x.upper() if isinstance(x, str) else x)
    
    #remove trailing and leading spaces
    obj_cols = list(df.select_dtypes('object'))
    df[obj_cols] = df[obj_cols].apply(lambda category: category.str.strip()) 
    
    #remove spaces in between values, replace with underscore
    df = df.map(lambda x: '_'.join(x.split()) if isinstance(x, str) else x)
    
    #convert to expected numerical datatypes
    float_cols = ['age','dp_ratio', 'down_payment_amount', 
                  'total_income','interest_rate','monthly_income']
    int_cols = ['bureau_score','res_years_at_current_city', 'loan_asset_cost','number_of_installments',
               'res_years_at_current_address','years_in_occupation']
    
    df[int_cols] = df[int_cols].astype(int)
    df[float_cols] = df[float_cols].astype(float)
    standard_df = df.copy()

    return standard_df

In [10]:
standard_data = standard_format_data('EDA_df.csv')
standard_data.to_csv(f'cleaned_{datafile_name}.csv')

In [11]:
standard_data.shape

(49681, 26)

At this point, the column names of the data has been lowercased, while all the categorical values have been capitalized. Spaces were converted to underscores. Data has been imputed; no missing values.

In [12]:
mixed_dtype_columns = [col for col in standard_data.columns if standard_data[col].apply(type).nunique() > 1]
print("Columns with mixed data types:", mixed_dtype_columns)

Columns with mixed data types: []


In [13]:
# #grouping asset_model
# text_list = ['SPLENDOR', 'ACCESS', 'BURGMAN', 'ACTIVA', 'NTORQ', 'JUPITER', 'HF', 'AVENIS']
# # Function to check and assign text or 'Others'
# def check_text(value):
#     value = value.replace('_', ' ')  # Replace underscores with spaces
#     for text in text_list:
#         if text in value:
#             return text
#     return 'Others'

# # Create new column based on the function
# df['asset_model_grp'] = df['asset_model'].apply(check_text)

In [14]:
#df['asset_model_keyword'].value_counts() #21225/52197 ##40% others; 60% keywords

In [15]:
 df['industry'].unique()

array(['Others', 'Apparels', 'Automobiles', 'Industrial equipment',
       'Agriculture', 'Healthcare Providers', 'Food & Beverages',
       'Durables', 'Healthcare', 'Computers', 'Food Products', nan,
       'Transportation Logistics', 'Education Institution',
       'Construction', 'Food Processing', 'Household Products',
       'Non Store Retail', 'Financial Services', 'Transport',
       'Hardware Equipment', 'Textiles', 'Insurance', 'Gen Indl Services',
       'Reliance Industries', 'Chemicals', 'Paper', 'Jewellery',
       'Broadline/ Multiline', 'Software', 'Electronics', 'FMCG',
       'Speciality'], dtype=object)

# Step 2: Data Wrangling for Training

In [16]:
#define a function that wrangles the base standard data
#i.e. grouping categories (crude feature engineering based on IV analysis)

def load_clean_data2(standard_df, segment):
    #load data
    basis_df = standard_df

    #add new target columns
    target_filename =  'tagged_12months_v2.csv'
    target_filepath = f'./Raw_Data/{target_filename}'
    
    new_col12m = pd.read_csv(target_filepath)
    new_col12m = new_col12m.rename(columns={'loan_customer_no':'loan_account_no'})

    merged_df_12m = pd.merge(basis_df, new_col12m, on='loan_account_no')
    full_df = merged_df_12m.copy()

    df = full_df.copy()
    #variable adjustments
    
    #manufacturer - Jan ok
    df['manufacturer_grp'] = np.where(
        (df['manufacturer'] == 'BAJAJ_AUTO') | (df['manufacturer'] == 'YAMAHA')
        | (df['manufacturer'] == 'APRILIA') | (df['manufacturer'] == 'ROYAL_ENFIELD')
        | (df['manufacturer'] == 'PIAGGIO') | (df['manufacturer'] == 'KTM')
        | (df['manufacturer'] == 'JAWA'),
        'OTHERS', df['manufacturer'])
    
    #channel - Jan ok
    df['channel_grp'] = np.where(
        (df['channel'] == 'ALTERNATE_CHANNEL') | (df['channel'] == 'BRANCH')
        | (df['channel'] == 'DIRECT')| (df['channel'] == 'REFERRAL'),
        'NON_DEALER', df['channel'])

    #industry - Jan ok
    df['industry_grp'] = np.where(
        (df['industry'] != 'APPARELS') & (df['industry'] != 'AGRICULTURE'),
        'OTHERS',df['industry'])

    #branch_name - Jan ok
    df['branch_grp'] = np.where(
        (df['branch_name'] == 'EAST_BANGALORE') | (df['branch_name'] == 'SOUTH_DELHI') |
        (df['branch_name'] == 'NORTH_BANGALORE') | (df['branch_name'] == 'EAST_DELHI') |
        (df['branch_name'] == 'NOIDA')| (df['branch_name'] == 'GURGAON')|
        (df['branch_name'] == 'WEST_BANGALORE')| (df['branch_name'] == 'NORTH_DELHI')
        | (df['branch_name'] == 'FARIDABAD'),
            'OTHERS',df['branch_name'])

    #grouping asset_model - Jan ok
    text_list = ['SPLENDOR', 'ACCESS', 'BURGMAN', 'ACTIVA', 'NTORQ', 'JUPITER', 'HF', 'AVENIS']
    # Function to check and assign text or 'Others'
    def check_text(value):
        value = value.replace('_', ' ')  # Replace underscores with spaces
        for text in text_list:
            if text in value:
                return text
        return 'OTHERS'

    # Create new column based on the function
    df['asset_model_grp'] = df['asset_model'].apply(check_text)

    #res_city - Jan oK
    df['res_city_grp'] = np.where(
        (df['res_city'] != 'MUMBAI') & (df['res_city'] != 'THANE')
        & (df['res_city'] != 'BANGALORE')& (df['res_city'] != 'DELHI')
        & (df['res_city'] != 'GHAZIABAD'),
        'OTHERS',df['res_city'])

    #res_state - Jan ok
    df['res_state_grp'] = np.where(
        (df['res_state'] == 'HARYANA') | (df['res_state'] == 'LAKSHADWEEP'),
        'OTHERS',df['res_state'])

    #res_resident_status - Jan ok
    df['res_resident_status_grp'] = np.where(
        (df['res_resident_status'] == 'OWNED_BY_PARENT_/SIBLING') | (df['res_resident_status'] == 'OWNED_BY_SELF/SPOUSE')
        , 'OWNED', df['res_resident_status'])

    df['res_resident_status_grp'] = np.where(
        (df['res_resident_status'] == 'COMPANY_PROVIDED') | (df['res_resident_status'] == 'RENTED')
        | (df['res_resident_status'] == 'HOSTEL')| (df['res_resident_status'] == 'PAYING_GUEST')
        , 'RENTED', df['res_resident_status_grp'])

    #branch_name
    df['nature_of_business'] = np.where(
        (df['nature_of_business'] != 'SERVICES') & (df['nature_of_business'] != 'AGRICULTURE'),
            'OTHERS',df['nature_of_business'])

    working_df = df.copy()
    working_df = working_df.drop(columns = ['manufacturer', 'channel','industry',
                                            'branch_name', 'asset_model','res_city',
                                            'res_state','res_resident_status'])

    #dfs
    if segment == 'NTC':
        df_NTC = working_df[working_df['bureau_score'] < 300]
        df_NTC = df_NTC.drop(columns = ['bureau_score'])
        return df_NTC
    
    elif segment == 'low_CIBIL':
        df_low_CIBIL = working_df[(working_df['bureau_score'] >= 300) & (working_df['bureau_score'] <= 730)]
        df_low_CIBIL = df_low_CIBIL.drop(columns = ['bureau_score'])
        return df_low_CIBIL
    
    elif segment == 'mid_CIBIL':
        df_mid_CIBIL = working_df[(working_df['bureau_score'] >= 731) & (working_df['bureau_score'] <= 749)]
        df_mid_CIBIL = df_mid_CIBIL.drop(columns = ['bureau_score'])
        return df_mid_CIBIL
    
    elif segment == 'high_CIBIL':
        df_high_CIBIL = working_df[working_df['bureau_score'] >= 750]
        df_high_CIBIL = df_high_CIBIL.drop(columns = ['bureau_score'])
        return df_high_CIBIL
        
    else:
        return ('Choose among NTC, low_CIBIL, mid_CIBIL, high_CIBIL')

In [17]:
wrangled_df_NTC = load_clean_data2(standard_data, 'NTC')
wrangled_df_low_CIBIL = load_clean_data2(standard_data, 'low_CIBIL')
wrangled_df_mid_CIBIL = load_clean_data2(standard_data, 'mid_CIBIL')
wrangled_df_high_CIBIL = load_clean_data2(standard_data, 'high_CIBIL')

wrangled_df_NTC.to_csv(f'./Wrangled_Data/wrangled_NTC.csv',index=False)
wrangled_df_low_CIBIL.to_csv(f'./Wrangled_Data/wrangled_low_CIBIL.csv',index=False)
wrangled_df_mid_CIBIL.to_csv(f'./Wrangled_Data/wrangled_mid_CIBIL.csv',index=False)
wrangled_df_high_CIBIL.to_csv(f'./Wrangled_Data/wrangled_high_CIBIL.csv',index=False)

In [18]:
print(wrangled_df_NTC.shape)
print(wrangled_df_low_CIBIL.shape)
print(wrangled_df_mid_CIBIL.shape)
print(wrangled_df_high_CIBIL.shape)

(18278, 26)
(11369, 26)
(7425, 26)
(12609, 26)


In [19]:
18278 + 11369 + 7425 + 12609

49681

In [20]:
basis_df.nature_of_business.unique()

array(['Services', 'Manufacturing', 'Others', 'Agriculture', nan,
       'Trading'], dtype=object)

In [21]:
for col in wrangled_df_NTC.columns:
    print(f'{col}')
    print(f'No. of unique values: {wrangled_df_NTC[col].nunique()}')
    print(f'No. of unique values: {wrangled_df_NTC[col].unique()} \n')

product_description
No. of unique values: 1
No. of unique values: ['TWO-WHEELER'] 

loan_account_no
No. of unique values: 18278
No. of unique values: ['TW01010102000002232820' 'TW01010104000002231508'
 'TW01030105000002242335' ... 'TW01030101000002280247'
 'TW01020103000002281621' 'TW01010107000002248785'] 

res_years_at_current_city
No. of unique values: 70
No. of unique values: [ 50  13  20  15  30  12  23   5   0   1  22  26  36   9  40  35   6  10
  31  34  24  28   2  32  44  54  49  16  27   8  37   7  25  21  48   4
  29  60  38  42  39  46   3  53  33  41  18  14  17  47  45  43  51  11
  62  56  57  88  19  55  59  58  52 208  75 201  63 244  65  70] 

age
No. of unique values: 8813
No. of unique values: [50.27197802 32.68406593 22.38736264 ... 38.81593407 46.65384615
 34.36538462] 

down_payment_amount
No. of unique values: 5153
No. of unique values: [39817. 10600. 45446. ... 11573.  9004. 41545.] 

total_income
No. of unique values: 418
No. of unique values: [ 960000.  36000